In [1]:
from pathlib import Path
import os
from datasets import load_dataset

In [7]:
CACHE_DIR = os.path.join(Path(os.getcwd()).resolve().parents[0] , "local_only", "data")

In [9]:
CACHE_DIR

'/mnt/f/chatbot_ui_v4/local_only/data'

In [10]:
os.makedirs(CACHE_DIR, exist_ok=True)
DATASET_NAME = "nlphuji/flickr30k"

In [11]:
def download_flickr30k_dataset(dataset_name, download_location=None):
    dataset = load_dataset(dataset_name, cache_dir=download_location)
    return dataset

In [12]:
dataset = download_flickr30k_dataset(dataset_name = DATASET_NAME, download_location = CACHE_DIR)

In [13]:
dataset

DatasetDict({
    test: Dataset({
        features: ['image', 'caption', 'sentids', 'split', 'img_id', 'filename'],
        num_rows: 31014
    })
})

In [14]:
data = dataset['test']

In [15]:
data

Dataset({
    features: ['image', 'caption', 'sentids', 'split', 'img_id', 'filename'],
    num_rows: 31014
})

In [16]:
data[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=333x500>,
 'caption': ['Two young guys with shaggy hair look at their hands while hanging out in the yard.',
  'Two young, White males are outside near many bushes.',
  'Two men in green shirts are standing in a yard.',
  'A man in a blue shirt standing in a garden.',
  'Two friends enjoy time spent together.'],
 'sentids': ['0', '1', '2', '3', '4'],
 'split': 'train',
 'img_id': '0',
 'filename': '1000092795.jpg'}

In [17]:
data[1]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x374>,
 'caption': ['Several men in hard hats are operating a giant pulley system.',
  'Workers look down from up above on a piece of equipment.',
  'Two men working on a machine wearing hard hats.',
  'Four men on top of a tall structure.',
  'Three men on a large rig.'],
 'sentids': ['5', '6', '7', '8', '9'],
 'split': 'train',
 'img_id': '1',
 'filename': '10002456.jpg'}

In [18]:
data[-1]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x375>,
 'caption': ['A man in shorts and a Hawaiian shirt leans over the rail of a pilot boat, with fog and mountains in the background.',
  'A young man hanging over the side of a boat, which is in a like with fog rolling over a hill behind it.',
  'A man is leaning off of the side of a blue and white boat as it sits in a body of water.',
  'A man riding a small boat in a harbor, with fog and mountains in the background.',
  'A man on a moored blue and white boat with hills and mist in the background.'],
 'sentids': ['155065', '155066', '155067', '155068', '155069'],
 'split': 'train',
 'img_id': '31013',
 'filename': '998845445.jpg'}

In [21]:
from pymongo import MongoClient, ASCENDING, UpdateOne

In [19]:
def get_mongo_db_client():
    client = MongoClient("mongodb://localhost:27017/")
    assert client.admin.command("ping") == {'ok': 1.0}
    return client

In [22]:
mongo_client  = get_mongo_db_client()

In [23]:
database = 'chatbot_ui_v4'

In [26]:
class FLICKR:
    def __init__(self, dataset_split):
        self.dataset_split = dataset_split
    def __iter__(self):
        for record in self.dataset_split:
            yield record
    def batch(self, batch_size):
        """
        Yield records in batches of batch_size
        """
        batch = []
        for record in self:
            batch.append(record)
            if len(batch) >= batch_size:
                yield batch
                batch = []
        
        if batch:
            yield batch  # Yield remaining records if any

In [64]:
def upload_data_to_mongo_db(dataset):
    mongo_db_client = get_mongo_db_client()
    
    database = mongo_db_client['chatbot_ui_v4']  

    caption_collection = database['caption']
    caption_collection.create_index([("sent_id", ASCENDING)], unique=True)

    #images_collection = database['images']
    #images_collection.create_index([("img_id", ASCENDING)], unique=True)

    iterator = FLICKR(dataset)
    print('Done with records:')
    batch_size = 32
    counter = 0
    for batch in iterator.batch(batch_size=batch_size):
        final_records = []
        for record in batch:
            to_append = [{'sent_id': sent_id, 'caption': caption} for sent_id, caption in zip(record['sentids'],record['caption'])]
            to_append = [dict(**i, img_id=record['img_id'], filename = record['filename']) for i in to_append]
            final_records.extend(to_append)
        
        records = [UpdateOne({'sent_id': i['sent_id']}, {'$set': i}, upsert=True) for i in final_records]
        caption_collection.bulk_write(records)

        counter+=batch_size
        if counter % (10*batch_size) == 0:
            print(counter, end='\t')


In [65]:
%%time

data = dataset['test']
upload_data_to_mongo_db(data)

Done with records:
320	640	960	1280	1600	1920	2240	2560	2880	3200	3520	3840	4160	4480	4800	5120	5440	5760	6080	6400	6720	7040	7360	7680	8000	8320	8640	8960	9280	9600	9920	10240	10560	10880	11200	11520	11840	12160	12480	12800	13120	13440	13760	14080	14400	14720	15040	15360	15680	16000	16320	16640	16960	17280	17600	17920	18240	18560	18880	19200	19520	19840	20160	20480	20800	21120	21440	21760	22080	22400	22720	23040	23360	23680	24000	24320	24640	24960	25280	25600	25920	26240	26560	26880	27200	27520	27840	28160	28480	28800	29120	29440	29760	30080	30400	30720	31040	CPU times: user 42.6 s, sys: 107 ms, total: 42.7 s
Wall time: 45.8 s


In [66]:
mongo_db_client = get_mongo_db_client()

In [67]:
#mongo_db_client['chatbot_ui_v4']['caption'].drop()
collection = mongo_db_client['chatbot_ui_v4']['caption']
collection.count_documents({})

155070

In [68]:
for i in collection.find().limit(2):
    print(i)

{'_id': ObjectId('686b569ddc158f44299de34d'), 'sent_id': '0', 'caption': 'Two young guys with shaggy hair look at their hands while hanging out in the yard.', 'filename': '1000092795.jpg', 'img_id': '0'}
{'_id': ObjectId('686b569ddc158f44299de34e'), 'sent_id': '1', 'caption': 'Two young, White males are outside near many bushes.', 'filename': '1000092795.jpg', 'img_id': '0'}


In [71]:
import torch
import open_clip
from PIL import Image

class OpenCLIPEmbedder:
    def __init__(self, model_name="ViT-L-14", pretrained="laion2b_s32b_b82k", device=None):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model, _, self.preprocess = open_clip.create_model_and_transforms(model_name, pretrained=pretrained)
        self.tokenizer = open_clip.get_tokenizer(model_name)
        self.model = self.model.to(self.device)
    
    def encode_text(self, texts, normalize=True):
        if isinstance(texts, str):
            texts = [texts]
        
        tokens = self.tokenizer(texts).to(self.device)
        with torch.no_grad():
            features = self.model.encode_text(tokens)
        if normalize:
            features /= features.norm(dim=-1, keepdim=True)
        return features.cpu().numpy()
    
    def encode_image(self, images, normalize=True):
        if isinstance(images, str):
            images = [Image.open(images)]
        elif isinstance(images, Image.Image):
            images = [images]
        
        images = torch.stack([self.preprocess(img) for img in images]).to(self.device)
        with torch.no_grad():
            features = self.model.encode_image(images)
        if normalize:
            features /= features.norm(dim=-1, keepdim=True)
        return features.cpu().numpy()


In [75]:
CHROMA_DB_PORT = '8010'
import chromadb
def get_chroma_db_client():
    client = chromadb.HttpClient(
        host="localhost",
        port=int(CHROMA_DB_PORT))
    return client

In [79]:
def upload_data_to_chroma_db(dataset, embed_model):
    vec_db_client = get_chroma_db_client()
    
    images_collection = vec_db_client.get_or_create_collection('images')

    iterator = FLICKR(dataset)
    print('Done with records:')
    batch_size = 64
    counter = 0
    for batch in iterator.batch(batch_size=batch_size):
        images = [i['image'] for i in batch]
        
        embeddings = embed_model.encode_image(images)
        documents = [i['filename'] for i in batch]
        ids = [i['img_id'] for i in batch]

        to_upload = dict(documents=documents, embeddings=embeddings, ids=ids)
        images_collection.add(**to_upload)
        
        counter+=batch_size
        if counter % (10*batch_size) == 0:
            print(counter, end='\t')


In [80]:
vec_db_client  = get_chroma_db_client()

In [81]:
embed_model = OpenCLIPEmbedder(device='cuda')

In [82]:
%%time

upload_data_to_chroma_db(dataset = dataset['test'], embed_model=embed_model)

Done with records:
640	1280	1920	2560	3200	3840	4480	5120	5760	6400	7040	7680	8320	8960	9600	10240	10880	11520	12160	12800	13440	14080	14720	15360	16000	16640	17280	17920	18560	19200	19840	20480	21120	21760	22400	23040	23680	24320	24960	25600	26240	26880	27520	28160	28800	29440	30080	30720	CPU times: user 29min 21s, sys: 6.3 s, total: 29min 27s
Wall time: 22min 42s


In [83]:
col = vec_db_client.get_collection('images')
col.count()

31014